In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.optimize import minimize

data = loadmat('ex5data1.mat')

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y', 'Xtest', 'ytest', 'Xval', 'yval'])
# 训练集
X_train,y_train=data['X'],data['y']
#(12,1) (12,1)
X_val,y_val=data['Xval'],data['yval']

X_test,y_test=data['Xtest'],data['ytest']

#输入变量插入偏置
X_train=np.insert(X_train,0,1,axis=1)
X_val=np.insert(X_val,0,1,axis=1)
X_test=np.insert(X_test,0,1,axis=1)

#linear regression 
#线性回归，代价函数，带正则化
def reg_cost(theta,X,y,lamda):
    cost=np.sum(np.power((X@theta-y.flatten()),2))
    reg=theta[1:]@theta[1:]*lamda
    return (cost+reg)/(2*len(X))
#参数初始化
theta=np.ones(X_train.shape[1])
lamda=1
loss=re_cost(theta,X_train,y_trian,lamda)  #输出代价loss


#梯度函数，带正则化
#输出代价函数的梯度
def reg_gradient(theta,X,y,lamda):
    grad=(X@theta-y.flatten())@X
    reg=lamda*theta
    reg[0]=0#第一项不参加正则化运算
    return (grad+reg)/(len(X))
reg_gradient(theta,X_train,y_train,lamda)


#利用给出的代价函数和梯度，更新theta,使得代价函数输出最小
def train_model(X,y,lamda):
    theta=np.ones(X.shape[1])
    res=minimize(fun=reg_cost,x0=theta,args=(X,y,lamda),method='TNC',jac=reg_gradient)
    return res.x  #训练，返回使得代价函数最小时的，theta值
theta_final = train_model(X_train,y_train,lamda=1)

plot_data()
plt.plot(X_train[:,1],X_train@theta_final,c='r')
plt.show()

#学习曲线
def plot_learning_curve(X_train,y_train,X_val,y_val,lamda):
    
    x = range(1,len(X_train)+1)
    training_cost = []		# 训练集的误差
    cv_cost = []			# 验证集的误差
    
    for i in x:        
        res = train_model(X_train[:i,:],y_train[:i,:],lamda) #训练集取样本个数 i
        
        #需要注意的是，当计算训练集、交叉验证集和测试集误差时，不计算正则项，所以令 λ=0
        training_cost_i = reg_cost(res,X_train[:i,:],y_train[:i,:],lamda = 0)
        cv_cost_i = reg_cost(res,X_val,y_val,lamda = 0) #验证集取自己全部样本
        training_cost.append(training_cost_i)   
        cv_cost.append(cv_cost_i)	
        			
    plt.plot(x,cv_cost,label = 'cv cost')
    plt.plot(x,training_cost,label = 'training cost')
    plt.legend()
    plt.xlabel('number of training examples')
    plt.ylabel('error')
    plt.show()
    
plot_learning_curve(X_train,y_train,X_val,y_val,lamda)

    #由图可知，当Jtrain和Jcv都很大时候，属于欠拟合问题，此时增加训练数据是没有用的，可以引入更多的相关特征来降低偏差。
#多项式回归
#进行特征映射获取特征多项式
def poly_feature(X,power):    
    for i in range(2,power+1):
        X = np.insert(X,X.shape[1],np.power(X[:,1],i),axis=1) # 在列尾插入
    return X
# 获得均值与方差
def get_mean_std(x):
    means = np.mean(x, axis=0) #按行取均值和方差
    std = np.std(x, axis=0)  
    return means,std
    
# 特征归一化
def feature_normalize(X,means,stds):    
    X[:,1:] = ( X[:,1:]  - means[1:]) / stds[1:]  # 第一列全为1，不用归一化
    return X

#获取特征多项式
power = 6

X_train_poly = poly_feature(X_train,power)# 获取特征多项式
X_val_poly = poly_feature(X_val,power)
X_test_poly = poly_feature(X_test,power)

train_means,train_stds = get_means_stds(X_train_poly)# 均值和方差

X_train_norm = feature_normalize(X_train_poly,train_means,train_stds)# 特征归一化
X_val_norm = feature_normalize(X_val_poly,train_means,train_stds)
X_test_norm = feature_normalize(X_test_poly,train_means,train_stds)

theta_fit = train_model(X_train_norm,y_train,lamda=0)# 训练新特征，获得新的优化模型参数

def plot_poly_fit():#绘制拟合曲线
    plot_data()#绘制原始数据
    
    x = np.linspace(-60,60,100)
    xx = x.reshape(100,1)
    xx = np.insert(xx,0,1,axis=1)
    xx = poly_feature(xx,power)
    xx = feature_normalize(xx,train_means,train_stds)#获取横坐标的特征多项式
    
    plt.plot(x, xx @ theta_fit,'r--')# xx @ theta_fit即为曲线的y值

plot_poly_fit()


SyntaxError: unexpected EOF while parsing (<ipython-input-1-8a113401a82c>, line 55)